## Supervised linear regression

Voor het product: 45 over het komende jaar

In [ ]:
import pandas as pd
import pyodbc
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
database = {'servername': 'localhost\SQLEXPRESS',
            'database': 'DATAWAREHOUSE'}
format = '%d-%m-%Y'

conn_str = ('DRIVER={SQL Server};'
            f'SERVER={database["servername"]};'
            f'DATABASE={database["database"]};'
            'Trusted_Connection=yes')

query = "SELECT * FROM dbo.sales_order"

#inladen naar df
try:
    with pyodbc.connect(conn_str) as conn:
        product_df = pd.read_sql_query(query, conn)
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# opsplitsen naar verschillende df's

orders_from_nw = product_df[product_df['product_id'].str.contains('NW')]
orders_from_aenc = product_df[product_df['product_id'].str.contains('AC')]
orders_from_aw = product_df[product_df['product_id'].str.contains('AW')]

orders_from_aenc


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

product_df

# NW

In [ ]:
import pandas as pd

def extract_year_month(df, date_column):
    df['year_month'] = df[date_column].dt.strftime('%Y-%m')
    return df


grouped_orders = orders_from_nw.groupby(['order_date']).agg({'id': 'count'})
sorted_orders = grouped_orders.reset_index().rename(columns={'id': 'order_count'})
sorted_orders = extract_year_month(sorted_orders, 'order_date')

sorted_orders.drop(['order_date'], axis=1, inplace=True)
sorted_orders = sorted_orders.groupby(['year_month']).agg({'order_count': 'sum'})
sorted_orders =  sorted_orders.reset_index()

orders = sorted_orders['order_count']
dates = sorted_orders['year_month']

values_dict = {dates[x]: orders[x] for x in range(len(orders) - 1)}
dates_2 = []
orders_2 = []
for k, v in values_dict.items():
    dates_2.append(k)
    orders_2.append(v)

values_dict = pd.DataFrame(values_dict, index=[0])

fin = {'year_month': dates_2, 'order_count': orders_2}
finished_df = pd.DataFrame(fin)
finished_df


In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

p, d, q = 1, 1, 1  
model = ARIMA(finished_df['order_count'], order=(p, d, q))
model_fit = model.fit()

forecast_steps = 1
future_dates = pd.date_range(start=pd.to_datetime('1998-05'), periods=forecast_steps, freq='M')
future_forecast = model_fit.forecast(steps=forecast_steps)

future_forecast_df = pd.DataFrame({'year_month': future_dates.strftime('%Y-%m'), 'order_count': int(future_forecast)})
finished_df = pd.concat([finished_df, future_forecast_df])

values_dict = {**values_dict, **{future_forecast_df.iloc[i]['year_month']: future_forecast_df.iloc[i]['order_count'] for i in range(len(future_forecast_df))}}
values_dict = pd.DataFrame(values_dict, index=[0])
values_dict


In [ ]:
import matplotlib.pyplot as plt

# Gegevens
dates = values_dict.columns
order_counts = values_dict.values[0]

# Maak de plot
plt.figure(figsize=(10, 6))
plt.plot(dates, order_counts, marker='o', linestyle='-')
plt.title('Voorspelde bestellingen voor NW')
plt.xlabel('Year-Month')
plt.ylabel('Order Count')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


2

#aenc

In [ ]:
import pandas as pd

def extract_year_month(df, date_column):
    df['year_month'] = df[date_column].dt.strftime('%Y-%m')
    return df

grouped_orders = orders_from_aenc.groupby(['order_date']).agg({'id': 'count'})
sorted_orders = grouped_orders.reset_index().rename(columns={'id': 'order_count'})
sorted_orders = extract_year_month(sorted_orders, 'order_date')

sorted_orders.drop(['order_date'], axis=1, inplace=True)
sorted_orders = sorted_orders.groupby(['year_month']).agg({'order_count': 'sum'})
sorted_orders =  sorted_orders.reset_index()

orders = sorted_orders['order_count']
dates = sorted_orders['year_month']

values_dict = {dates[x]: orders[x] for x in range(len(orders)-2)}
dates_2 = []
orders_2 = []
for k, v in values_dict.items():
    dates_2.append(k)
    orders_2.append(v)

values_dict = pd.DataFrame(values_dict, index=[0])
fin = {'year_month': dates_2, 'order_count': orders_2}
finished_df = pd.DataFrame(fin)
finished_df



In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

p, d, q = 1, 1, 1 
model = ARIMA(finished_df['order_count'], order=(p, d, q))
model_fit = model.fit()

forecast_steps = 1
future_dates = pd.date_range(start=pd.to_datetime('1998-12'), periods=forecast_steps, freq='M')
future_forecast = model_fit.forecast(steps=forecast_steps)

future_forecast_df = pd.DataFrame({'year_month': future_dates.strftime('%Y-%m'), 'order_count': int(future_forecast)})
finished_df = pd.concat([finished_df, future_forecast_df])

values_dict = {**values_dict, **{future_forecast_df.iloc[i]['year_month']: future_forecast_df.iloc[i]['order_count'] for i in range(len(future_forecast_df))}}
values_dict = pd.DataFrame(values_dict, index=[0])
values_dict



In [ ]:
import matplotlib.pyplot as plt

# Gegevens
dates = values_dict.columns
order_counts = values_dict.values[0]

# Maak de plot
plt.figure(figsize=(10, 6))
plt.plot(dates, order_counts, marker='o', linestyle='-')
plt.title('Voorspelde bestellingen voor AENC')
plt.xlabel('Year-Month')
plt.ylabel('Order Count')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


#aw

In [ ]:
import pandas as pd

def extract_year_month(df, date_column):
    df['year_month'] = df[date_column].dt.strftime('%Y-%m')
    return df

grouped_orders = orders_from_aw.groupby(['order_date']).agg({'id': 'count'})
sorted_orders = grouped_orders.reset_index().rename(columns={'id': 'order_count'})

sorted_orders = extract_year_month(sorted_orders, 'order_date')

sorted_orders.drop(['order_date'], axis=1, inplace=True)
sorted_orders = sorted_orders.groupby(['year_month']).agg({'order_count': 'sum'})
sorted_orders =  sorted_orders.reset_index()
sorted_orders

orders = sorted_orders['order_count']
dates = sorted_orders['year_month']

values_dict = {dates[x]: orders[x] for x in range(len(orders))}
dates_2 = []
orders_2 = []
for k, v in values_dict.items():
    dates_2.append(k)
    orders_2.append(v)

values_dict = pd.DataFrame(values_dict, index=[0])

fin = {'year_month': dates_2, 'order_count': orders_2}
finished_df = pd.DataFrame(fin)
finished_df



In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

p, d, q = 1, 1, 1 
model = ARIMA(finished_df['order_count'], order=(p, d, q))
model_fit = model.fit()

forecast_steps = 1
future_dates = pd.date_range(start=pd.to_datetime('2014-07'), periods=forecast_steps, freq='M')
future_forecast = model_fit.forecast(steps=forecast_steps)

future_forecast_df = pd.DataFrame({'year_month': future_dates.strftime('%Y-%m'), 'order_count': int(future_forecast)})
finished_df = pd.concat([finished_df, future_forecast_df])

values_dict = {**values_dict, **{future_forecast_df.iloc[i]['year_month']: future_forecast_df.iloc[i]['order_count'] for i in range(len(future_forecast_df))}}

values_dict = pd.DataFrame(values_dict, index=[0])
values_dict


grafiekje bruv

In [ ]:
import matplotlib.pyplot as plt

# Gegevens
dates = values_dict.columns
order_counts = values_dict.values[0]

# Maak de plot
plt.figure(figsize=(10, 6))
plt.plot(dates, order_counts, marker='o', linestyle='-')
plt.title('Voorspelde bestellingen voor AW')
plt.xlabel('Year-Month')
plt.ylabel('Order Count')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# Verwachtingen per kwartaal

In [ ]:
import statsmodels.api as sm

grouped_nw = orders_from_nw.groupby(['order_date']).agg({'id': 'count'})
grouped_aw = orders_from_aw.groupby(['order_date']).agg({'id': 'count'})
grouped_ac = orders_from_aenc.groupby(['order_date']).agg({'id': 'count'})

all = [grouped_nw, grouped_aw, grouped_ac]

In [ ]:
names = ['nw', 'aw', 'ac']
values_dict = {}
count = 0
for data in all:
    sorted_orders = data.reset_index().rename(columns={'id': 'order_count'})
    sorted_orders['order_date'] = pd.to_datetime(sorted_orders['order_date'])
    df = sorted_orders

    df_grouped = df.groupby(pd.Grouper(key='order_date', freq='Q')).sum().reset_index()
    df_grouped['year'] = df_grouped['order_date'].dt.year
    df_grouped['quarter'] = df_grouped['order_date'].dt.quarter
    df_grouped = df_grouped.drop(columns=['order_date'])
    df_grouped = df_grouped[['year', 'quarter', 'order_count']]

    df_sorted = df_grouped.sort_values(by=['year', 'quarter'])
    values_dict['values_' + names[count]] = df_sorted
    count += 1

In [ ]:
nw_values = values_dict['values_nw'].copy()
aw_values = values_dict['values_aw'].copy()
ac_values = values_dict['values_ac'].copy()

frames = [aw_values, nw_values, ac_values]
aw_values


In [ ]:
df = aw_values.copy()
df['date'] = pd.to_datetime(df['year'].astype(str) + 'Q' + df['quarter'].astype(str))

df.set_index('date', inplace=True)
df.drop(columns=['year', 'quarter'], inplace=True)

model = sm.tsa.ARIMA(df['order_count'], order=(1, 1, 1))  # You may need to tune the order parameters
model_fit = model.fit()

forecast = model_fit.forecast(steps=1)
prediction = int(forecast)

last_date = df.index[-1]
next_date = (last_date + pd.offsets.QuarterEnd()).normalize()
n_year = int(next_date.year)
n_quarter = int(next_date.quarter) + 1

aw_values = aw_values._append({'year': n_year, 'quarter':n_quarter, 'order_count': prediction}, ignore_index=True)
aw_values
